Python Version of the QL Bermudan Swaption Example

In [1]:
import QuantLib as ql

In [2]:
todaysDate = ql.Date(15,2,2002)
calendar = ql.TARGET()
settlementDate = ql.Date(19,2,2002)
ql.Settings.instance().evaluationDate = todaysDate

flat yield term structure impling 1x5 swap at 5%

In [3]:
flatRate = ql.SimpleQuote(0.04875825)
rhTermStructure = ql.YieldTermStructureHandle(ql.FlatForward(settlementDate,ql.QuoteHandle(flatRate),ql.Actual365Fixed()))

Define the ATM/OTM/ITM swaps

In [4]:
fixedLegFrequency = ql.Annual
fixedLegConvention = ql.Unadjusted
floatingLegConvention = ql.ModifiedFollowing
fixedLegDayCounter = ql.Thirty360(ql.Thirty360.European)
floatingLegFrequency = ql.Semiannual
swapType = ql.VanillaSwap.Payer
dummyFixedRate = 0.03
indexSixMonths = ql.Euribor6M(rhTermStructure)

In [5]:
startDate = calendar.advance(settlementDate, ql.Period(1,ql.Years),floatingLegConvention)
maturity = calendar.advance(settlementDate, ql.Period(5,ql.Years),floatingLegConvention)
fixedSchedule = ql.Schedule(startDate,maturity,ql.Period(fixedLegFrequency),calendar,fixedLegConvention,fixedLegConvention,
                           ql.DateGeneration.Forward,False)
floatSchedule = ql.Schedule(startDate,maturity,ql.Period(floatingLegFrequency),calendar,floatingLegConvention,floatingLegConvention,
                           ql.DateGeneration.Forward,False)

In [6]:
swap = ql.VanillaSwap(swapType,1000.0,
            fixedSchedule, dummyFixedRate, fixedLegDayCounter,
            floatSchedule, indexSixMonths, 0.0,
            indexSixMonths.dayCounter())

In [7]:
swap.setPricingEngine(ql.DiscountingSwapEngine(rhTermStructure))

In [8]:
swap.NPV()

67.56094636176408

In [9]:
fixedATMRate = swap.fairRate()
fixedOTMRate = fixedATMRate * 1.2
fixedITMRate = fixedATMRate * 0.8

In [10]:
atmSwap = ql.VanillaSwap(swapType,1000.0,
            fixedSchedule, fixedATMRate, fixedLegDayCounter,
            floatSchedule, indexSixMonths, 0.0,
            indexSixMonths.dayCounter())
otmSwap = ql.VanillaSwap(swapType,1000.0,
            fixedSchedule, fixedOTMRate, fixedLegDayCounter,
            floatSchedule, indexSixMonths, 0.0,
            indexSixMonths.dayCounter())
itmSwap = ql.VanillaSwap(swapType,1000.0,
            fixedSchedule, fixedITMRate, fixedLegDayCounter,
            floatSchedule, indexSixMonths, 0.0,
            indexSixMonths.dayCounter())

In [11]:
atmSwap.setPricingEngine(ql.DiscountingSwapEngine(rhTermStructure))

In [12]:
atmSwap.NPV()

-1.1368683772161603e-13

In [13]:
swaptionMaturities = [ql.Period(i, ql.Years) for i in range(1,6)]

In [14]:
numRows = 5
numCols = 5
swapLenghts = [      1,     2,     3,     4,     5]
swaptionVols = [
  0.1490, 0.1340, 0.1228, 0.1189, 0.1148,
  0.1290, 0.1201, 0.1146, 0.1108, 0.1040,
  0.1149, 0.1112, 0.1070, 0.1010, 0.0957,
  0.1047, 0.1021, 0.0980, 0.0951, 0.1270,
  0.1000, 0.0950, 0.0900, 0.1230, 0.1160]

In [15]:
swaptions = []
#List of times that have to be included in the timegrid
times = []
for i in range(0,numRows):
    j = numCols - i -1 #1x5, 2x4, 3x3, 4x2, 5x1
    k = i*numCols + j
    vol = ql.SimpleQuote(swaptionVols[k])
    swaptions += [ql.SwaptionHelper(swaptionMaturities[i],
                               ql.Period(swapLenghts[j], ql.Years),
                               ql.QuoteHandle(vol),
                               indexSixMonths,
                               indexSixMonths.tenor(),
                               indexSixMonths.dayCounter(),
                               indexSixMonths.dayCounter(),
                               rhTermStructure)]
    times+=[t for t in swaptions[-1].times()]

Building time-grid

In [16]:
grid = ql.TimeGrid(times, 30)

defining the models

In [17]:
modelG2 = ql.G2(rhTermStructure)
modelHW = ql.HullWhite(rhTermStructure)
modelHW2 = ql.HullWhite(rhTermStructure)
modelBK = ql.BlackKarasinski(rhTermStructure)

model calibrations

In [18]:
print("G2 (analytic formulae) calibration")
for s in swaptions:
    s.setPricingEngine(ql.G2SwaptionEngine(modelG2, 6.0, 16))

G2 (analytic formulae) calibration


In [19]:
def calibrateModel(model, helpers):
    om = ql.LevenbergMarquardt()
    model.calibrate(helpers, om, ql.EndCriteria(400, 100, 1.0e-8, 1.0e-8, 1.0e-8))
    for i in range(0,numRows):
        j = numCols - i -1 # 1x5, 2x4, 3x3, 4x2, 5x1
        k = i*numCols + j;
        npv = helpers[i].modelValue()
        implied = helpers[i].impliedVolatility(npv, 1e-4, 1000, 0.05, 0.50)
        diff = implied - swaptionVols[k]
        print('{:<1}{:<1}{:<1}: model {:<7}, market {:<7} ({:<7})'.format(str(i+1), ("x"), str(swapLenghts[j]),("%.6f"%implied),
                                                                 ("%.6f"%swaptionVols[k]),("%.6f"%diff)))

In [20]:
calibrateModel(modelG2, swaptions)

1x5: model 0.100455, market 0.114800 (-0.014345)
2x4: model 0.105123, market 0.110800 (-0.005677)
3x3: model 0.107050, market 0.107000 (0.000050)
4x2: model 0.108382, market 0.102100 (0.006282)
5x1: model 0.109439, market 0.100000 (0.009439)


In [21]:
print( "calibrated to:")
print( "a     =" ,"%.6f"%modelG2.params()[0])
print( "sigma =" ,"%.6f"%modelG2.params()[1])
print( "b     =" ,"%.6f"%modelG2.params()[2])
print( "eta   =" ,"%.6f"%modelG2.params()[3])
print( "rho   =" ,"%.6f"%modelG2.params()[4])

calibrated to:
a     = 0.050103
sigma = 0.009450
b     = 0.050101
eta   = 0.009450
rho   = -0.763305


In [22]:
print("Hull-White (analytic formulae) calibration")

Hull-White (analytic formulae) calibration


In [23]:
for s in swaptions:
    s.setPricingEngine(ql.JamshidianSwaptionEngine(modelHW))

In [24]:
calibrateModel(modelHW, swaptions)

1x5: model 0.106204, market 0.114800 (-0.008596)
2x4: model 0.106296, market 0.110800 (-0.004504)
3x3: model 0.106341, market 0.107000 (-0.000659)
4x2: model 0.106443, market 0.102100 (0.004343)
5x1: model 0.106613, market 0.100000 (0.006613)


In [25]:
print( "calibrated to:")
print( "a     =" ,"%.6f"%modelHW.params()[0])
print( "sigma =" ,"%.6f"%modelHW.params()[1])

calibrated to:
a     = 0.046414
sigma = 0.005869


In [26]:
print("Hull-White (numerical) calibration")

Hull-White (numerical) calibration


In [27]:
for s in swaptions:
    s.setPricingEngine(ql.TreeSwaptionEngine(modelHW2, grid))

In [28]:
calibrateModel(modelHW2, swaptions)

1x5: model 0.103119, market 0.114800 (-0.011681)
2x4: model 0.105462, market 0.110800 (-0.005338)
3x3: model 0.106691, market 0.107000 (-0.000309)
4x2: model 0.107402, market 0.102100 (0.005302)
5x1: model 0.107972, market 0.100000 (0.007972)


In [29]:
print( "calibrated to:")
print( "a     =" ,"%.6f"%modelHW2.params()[0])
print( "sigma =" ,"%.6f"%modelHW2.params()[1])

calibrated to:
a     = 0.055229
sigma = 0.006106


In [30]:
print("Black-Karasinski (numerical) calibration")

Black-Karasinski (numerical) calibration


In [31]:
for s in swaptions:
    s.setPricingEngine(ql.TreeSwaptionEngine(modelBK, grid))

In [32]:
calibrateModel(modelBK, swaptions)

1x5: model 0.103259, market 0.114800 (-0.011541)
2x4: model 0.105657, market 0.110800 (-0.005143)
3x3: model 0.106786, market 0.107000 (-0.000214)
4x2: model 0.107368, market 0.102100 (0.005268)
5x1: model 0.107779, market 0.100000 (0.007779)


In [33]:
print( "calibrated to:")
print( "a     =" ,"%.6f"%modelBK.params()[0])
print( "sigma =" ,"%.6f"%modelBK.params()[1])

calibrated to:
a     = 0.043389
sigma = 0.120746


ATM Bermudan swaption pricing

In [34]:
print("Payer bermudan swaption struck at %.6f (ATM)" % fixedATMRate)

Payer bermudan swaption struck at 0.050007 (ATM)


In [35]:
leg = swap.fixedLeg()

In [36]:
bermudanDates = [d for d in fixedSchedule]

In [37]:
bermudanExercise = ql.BermudanExercise(bermudanDates)

In [38]:
bermudanSwaption = ql.Swaption(atmSwap, bermudanExercise)

Do the pricing for each model

G2 price the European swaption here, it should switch to bermudan

In [39]:
def calc_swaption_npv (swaption):
    try:
        swaption.setPricingEngine(ql.TreeSwaptionEngine(modelG2, 50))
        print ("G2 (tree):      %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.FdG2SwaptionEngine(modelG2))
        print ("G2 (fdm):      %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.TreeSwaptionEngine(modelHW, 50))
        print ("HW (tree):      %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.FdHullWhiteSwaptionEngine(modelHW))
        print ("HW (fdm):       %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.TreeSwaptionEngine(modelHW2, 50))
        print ("HW (num, tree): %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.FdHullWhiteSwaptionEngine(modelHW2))
        print ("HW (num, fdm):  %.6f" % swaption.NPV())
    except:
        pass
    try:
        swaption.setPricingEngine(ql.TreeSwaptionEngine(modelBK, 50))
        print ("BK:             %.6f" % swaption.NPV())
    except:
        pass

In [40]:
calc_swaption_npv(bermudanSwaption)

G2 (tree):      10.983217
G2 (fdm):      11.041164
HW (tree):      10.048167
HW (fdm):       10.059288
HW (num, tree): 10.273781
HW (num, fdm):  10.282544
BK:             10.124047


OTM Bermudan swaption pricing

In [41]:
print("Payer bermudan swaption struck at %.6f (OTM)" % fixedOTMRate)

Payer bermudan swaption struck at 0.060009 (OTM)


In [42]:
otmBermudanSwaption =ql.Swaption(otmSwap,bermudanExercise)

In [43]:
calc_swaption_npv(otmBermudanSwaption)

G2 (tree):      2.261356
G2 (fdm):      2.173593
HW (tree):      1.654076
HW (fdm):       1.645590
HW (num, tree): 1.771916
HW (num, fdm):  1.750425
BK:             2.208262


ITM Bermudan swaption pricing

In [44]:
print("Payer bermudan swaption struck at %.6f (ITM)" % fixedITMRate)

Payer bermudan swaption struck at 0.040006 (ITM)


In [45]:
itmBermudanSwaption =ql.Swaption(itmSwap,bermudanExercise)

In [46]:
calc_swaption_npv(itmBermudanSwaption)

G2 (tree):      34.904474
G2 (fdm):      34.845978
HW (tree):      34.450006
HW (fdm):       34.478416
HW (num, tree): 34.529802
HW (num, fdm):  34.556588
BK:             34.214183
